In [ ]:
import numpy as np
import scipy.io
from midiutil import MIDIFile

In [ ]:
key_time = 50e-3*0.5
Fs = 48000

In [ ]:
keys = scipy.io.loadmat('MATLAB_data/31Jan_jc.mat')['key']
notes_list = scipy.io.loadmat('notes_88.mat')['notes']
freq_list = scipy.io.loadmat('freq_480.mat')['f_cropped']
#freq_list = scipy.io.loadmat('freq_441.mat')['f_cropped']
notes_list = notes_list.reshape(np.size(notes_list))
freq_list = freq_list.reshape(np.size(freq_list))

In [ ]:
print(np.shape(notes_list))
print(np.shape(freq_list))
print(np.shape(keys))


In [ ]:
def get_intensity(cur_key,cur_freq,wave_freq):
    if cur_key == 1:
        sigma = notes_list[1]-notes_list[0]
    else:
        sigma = notes_list[cur_key-1]-notes_list[cur_key-2]
    sigma = sigma/2
    intensity = np.exp(-0.5 *((cur_freq-wave_freq)/sigma)**2)
    #print('cur', cur_freq,'wave',wave_freq)
    return intensity

In [ ]:
def picknotes(cur_key,cur_freq):
    j = 0
    while j < len(cur_key):
        if cur_key[j] != 0:
            
            d = {}
            d['note'] = cur_key[j]
            d['start'] = j
            while cur_key[j] != 0 and j < len(cur_key)-1:
                j += 1
            d['end'] = j - 1
            intensity = get_intensity(cur_key[j-1],cur_freq,notes_list[cur_key[j-1]-1])
            d['intensity'] = intensity
            notes.append(d)
        j += 1
# picknotes(keys[500])     
# print(notes)      


In [ ]:
notes = []
for i in range(np.shape(keys)[0]):
    cur_key = keys[i]
    cur_freq = freq_list[i]
    picknotes(cur_key,cur_freq)
    


In [ ]:
notes = sorted(notes, key = lambda notes: notes["start"])
notes = sorted(notes, key = lambda notes: -notes["note"])

In [ ]:
print(notes)

In [ ]:
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track
time = 0    # start at the beginning
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, 1200)

In [ ]:
for note in notes:
    mf.addNote(track,1,note['note']+20, note['start'], note['end']-note['start']+1,100)


In [ ]:
#duration = [[1e9,0]]*88
def merge(notes):
    duration = [0]*88 #[start, end]
    checker = [0]*88
    for note in notes:
        if checker[note['note']-1] == 0:
            checker[note['note']-1] = 1
            duration[note['note']-1]=[]
            duration[note['note']-1].append([note['start'], note['end']])
        else:
            connected = False
            current_note_list = duration[note['note']-1]
            #print(current_note_list)
            for num, current_note in enumerate(current_note_list):
                if (current_note[1]+1) >= note['start'] and current_note[0] <= note['end']:
                    connected = True
                    current_note[0] = min(note['start'],current_note[0])


                    if note['end'] > current_note[1]:
                        
                        i = num + 1
                        while i < len(current_note_list) and note['end'] > current_note_list[i][1]:
                            i += 1
                        current_note[1] = max(note['end'],current_note_list[i-1][1])
                        for j in range(num+1,i):
                            del current_note_list[j]
                                
                    break
            if not connected:
                duration[note['note']-1].append([note['start'], note['end']])
                duration[note['note']-1].sort()
    return duration
        #1.check if connect with existed
        #2.check if multiple existed can be combined
        #3.if not, add new (in time order!!)


In [ ]:
def convert(duration):
    notes = []
    for key, l in enumerate(duration):
        if l != 0:
            for item in l:
                #print(item[0])
                notes.append({'note': key+1, 'start': item[0], 'end': item[1]})
    return notes

In [ ]:
d = merge(notes)
new_notes=convert(d)

In [ ]:
for note in new_notes:
    # if note['end']-note['start']+1 > maxi:
    #     maxi = note['end']-note['start']+1
    #mf.addNote(track,channel[note['note']-1],note['note'], note['start'], note['end']-note['start']+1,100)
    #if note['end']== note['start']:
    #    continue
    mf.addNote(track,0,note['note']+20, note['start'], note['end']-note['start']+1,100)
    #print(note['end']-note['start']+1)

In [ ]:
with open("midifile/31Jan_jc_int.mid", 'wb') as outf:
   mf.writeFile(outf)

In [ ]:
print(len(new_notes))

In [ ]:
for note in notes:
    if note['note']==76:
        print(note)

In [ ]:
for note in new_notes:
    if note['note']==76:
        print(note)